<a href="https://github.com/Tim55667757/TKSBrokerAPI"><img alt="TKSBrokerAPI-Logo" src="https://raw.githubusercontent.com/Tim55667757/TKSBrokerAPI/develop/docs/media/TKSBrokerAPI-Logo.png" width="1200"/></a>

# Как быстро найти аномалии в числовых рядах с помощью метода Хампеля

**Авторы:** [Тимур Гильмуллин](https://www.linkedin.com/in/tgilmullin), к.т.н., [Мансур Гильмуллин](http://www.linkedin.com/in/mgilmullin), к.п.н.

**Читайте обзорную статью по теме:** https://forworktests.blogspot.com/2022/12/blog-post.html

Для работы с примерами в данном ноутбуке вам понадобятся `python >= 3.9` и установленные в него 3-rd party пакеты:
- **[PriceGenerator >= 1.2.74](https://github.com/Tim55667757/PriceGenerator/blob/master/README_RU.md)** — этот пакет умеет генерировать временные ряды с данными, похожими на случайные биржевые цены;
- **[TKSBrokerAPI >= 1.6.dev136](https://github.com/Tim55667757/TKSBrokerAPI/blob/develop/README.md)** — этот пакет содержит в себе библиотеку [TradeRoutines](https://github.com/Tim55667757/TKSBrokerAPI/blob/develop/tksbrokerapi/TradeRoutines.py), в которой есть необходимые методы для анализа числовых рядов.

In [1]:
# Установим указанные выше pip-пакеты в текущий Jupyter kernel:
import sys

import bokeh.plotting

!{sys.executable} -m pip install PriceGenerator==1.2.74
!{sys.executable} -m pip install TKSBrokerAPI==1.6.dev136

In [2]:
# Импортируем все необходимые для работы ноутбука и примеров библиотеки:
from datetime import datetime, timedelta
import pandas as pd

from pricegenerator.PriceGenerator import PriceGenerator, uLogger
from tksbrokerapi.TradeRoutines import HampelFilter, HampelAnomalyDetection

## Введение

На практике очень часто встречаются задачи, для решения которых требуется найти аномалии в числовых рядах. Для простоты понимания можно считать, что это такие значения, которые отличаются от большинства чисел в ряде по некоторым признакам (выброс, нестандартное значение или отклонение от нормы). Такие задачи встречаются в различных областях:

- очистка зашумлённых данных в датасайнс ([Data Science](https://en.wikipedia.org/wiki/Data_science));
- фильтрация выбросов в обучающей выборке для нейросетей в машинном обучении ([Machine Learning](https://en.wikipedia.org/wiki/Machine_learning));
- поиск аномальной сетевой хакерской активности, при мониторинге трафика и событий в кибербезопасности ([Cybersecurity](https://en.wikipedia.org/wiki/Cyber_security);
- выявление выбросов или хвостов в потоке биржевых данных в алгоритмической торговле ([Algorithmic Trading](https://en.wikipedia.org/wiki/Algorithmic_trading));
- а также в любых задачах на поиск аномалий, где данные могут быть представлены в виде числового ряда.

Понятия числового ряда в математическом анализе и в статистике отличаются. Мы принимаем под числовым рядом его статистическое понимание, то есть конечную последовательность чисел (аналог выборки). Далее разберём различные толкования аномалии и примеры, как быстро и эффективно найти аномалии в числовых рядах с помощью модифицированного **метода Хампеля** (Hampel F.R.).

## Понятие аномалии числового ряда

Традиционно наиболее полезными оценками для характеристики вариативности данных считаются выборочное среднее ([sample mean](https://en.wikipedia.org/wiki/Sample_mean_and_covariance)) и дисперсия ([variance](https://en.wikipedia.org/wiki/Variance)) выборки. Они дают хорошую оценку местоположения и разброса данных в выборке, но только если выбросы её «не загрязняют». Даже если данные содержат только одно наблюдение, которое значительно отличается от остальных в выборке, то выборочное среднее может также значительно отклоняться от среднего в выборке без этого выброса.

Чтобы измерить устойчивость выбранного метода статистической оценки к выбросам, Хампель ввёл понятие точки разрыва ([breakdown point](https://en.wikipedia.org/wiki/Robust_statistics#Breakdown_point)). **_Точка разрыва_** — это наибольший процент «загрязнённых данных» (доля неправильных наблюдений или выбросов), которые выбранный метод может обработать, прежде чем начнёт выдавать неверный результат. Это могут быть произвольно большие аберрантные (отклоняющиеся от нормы, ошибочные) значения. Очевидно, что точка разрыва не может превышать 50%. Чтобы более надёжно оценивать местоположение и разброс элементов выборки, часто рекомендуют комбинировать медиану и среднее абсолютное отклонение ([MAD, Median Absolute Deviation](https://en.wikipedia.org/wiki/Median_absolute_deviation)). Именно они лежат в основе метода фильтрации выборки на выбросы по Хампелю [1].

Значение **MAD** вычисляется так:

( 1 )   MAD (X) = Median ( | x1 − Median (X) | , ..., | xn − Median (X) | ), где X — выборка из n наблюдений x1, ..., xn.

По Хампелю, под **_выбросом ([outlier](https://en.wikipedia.org/wiki/Outlier)) выборки_** понимается такое число x из выборки X, модуль разности которого и медианы выборки больше, чем MAD выборки, вычисленное по формуле (1) и умноженное на специальный коэффициент k ([scale factor](https://en.wikipedia.org/wiki/Scale_parameter#Estimation)), зависящий от распределения (≈1.4826 для нормального) [2].

На практике для построения фильтра выбросов Хампеля это определение модифицируется с использованием скользящих окон ([sliding windows](https://www.geeksforgeeks.org/window-sliding-technique/)) и параметра s ([sigma](https://en.wikipedia.org/wiki/Standard_deviation)), порогового количества стандартных отклонений. Это значит, что медиана и MAD вычисляются для всех скользящих окон, состоящих из элементов выборки. Далее все модули разности элементов выборки и медианы сравниваются с MAD, умноженный на коэффициент k и умноженный на параметр s [3]. Более высокий порог s стандартного отклонения делает фильтр более щадящим, более низкий порог идентифицирует больше чисел как выбросы.

Теперь определимся с основным понятием: что мы будем понимать под аномалией в конечном числовом ряду для практических задач.

**Определение 1.** Под **аномалией (anomaly) числового ряда** понимается такое число x из ряда X, которое по модулю отличается от медианы скользящего окна более чем в s раз от MAD этого окна, умноженного на коэффициент k.

Для того чтобы максимально обобщить фильтр аномалий и абстрагировать его от решаемой практической задачи, предлагаем следующее теоретико-множественное толкование.

Все аномалии ряда образуют его некоторое подмножество A:

( 2 )   A = { a | | a − Median (Xi) | > s ∙ k ∙ MAD (Xi) }, где a — аномальный элемент из X, вычисленный согласно определению 1, Х — исходный числовой ряд, Xi — ряд чисел i-го скользящего окна, всего окон: ( n − w + 1 ), где w — размер окна.

Фильтр аномалий числового ряда определим как функцию:

( 3 )   F: X → { True, False }. F (xi) = True, если xi ∈ A; False, если xi ∉ A. Здесь Х — исходный числовой ряд, состоящий из n элементов xi, A — множество аномалий (2).

## Фильтрация числового ряда на аномалии методом Хампеля

Для фильтрации числового ряда на наличие в нём аномалий, предлагается использовать Python-реализацию функции [`HampelFilter()`](https://tim55667757.github.io/TKSBrokerAPI/docs/tksbrokerapi/TradeRoutines.html#HampelFilter) из библиотеки [`TKSBrokerAPI`](https://github.com/Tim55667757/TKSBrokerAPI). Эта функция позволяет обнаружить аномалию среди значений любого числового ряда, используя метод фильтрации по Хампелю. Фильтр обнаруживает аномалии, определяемые согласно (2).

Настраиваемые параметры: `window` (переменная w из формул выше) — размер скользящего окна (по умолчанию 5), `sigma` (переменная s) — пороговое количество стандартных отклонений (по умолчанию 3), `scaleFactor` (переменная k) — специальный коэффициент, зависящий от распределения ряда (по умолчанию 1.4826).

Функция фильтрации `HampelFilter()` возвращает новый ряд F, согласно (3) состоящий из значений `True` или `False`, где `True` означает наличие аномального элемента на соответствующей позиции в исходном ряду.

Будет проще понять как работает эта функция и что считает за аномалии на конкретных примерах. Сначала рассмотрим фильтрацию числовых рядов со значениями параметров по умолчанию:

In [3]:
testData1 = [
    pd.Series([10, 10, 10, 10, 10]),  # [False, False, False, False, False], здесь все элементы равны, нет ни одного "подозрительного" элемента, значит и аномалий нет.
    pd.Series([1, 10, 10, 10, 10]),  # [True, False, False, False, False], здесь первый элемент явно выбивается из общего ряда, поэтому он аномальный.
    pd.Series([1, 5, 10, 10, 10]),  # [True, True, False, False, False], в этом ряду больше элементов равных 10, первые два выбиваются из общей картины, значит они оба аномальные.
    pd.Series([1, 5, 1, 1, 1]),  # [False, True, False, False, False], аналогично, как во втором примере, второй элемент аномальный, потому что отличается от большинства в ряду.
]
for i, test in enumerate(testData1):
    print("Input series {}:  {}".format(i + 1, list(test)))
    print("Output series {}: {}".format(i + 1, list(HampelFilter(test))))

Input series 1:  [10, 10, 10, 10, 10]
C:\python3\envs\py39\lib\site-packages\tksbrokerapi\TradeRoutines.py:330: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  new = new[window: -window]
Output series 1: [False, False, False, False, False]
Input series 2:  [1, 10, 10, 10, 10]
Output series 2: [True, False, False, False, False]
Input series 3:  [1, 5, 10, 10, 10]
Output series 3: [True, True, False, False, False]
Input series 4:  [1, 5, 1, 1, 1]
Output series 4: [False, True, False, False, False]


По умолчанию размер скользящего окна равен 5, то есть совпадал с длиной ряда в примере выше. Давайте посмотрим, что изменится, если установить размер скользящего окна равным 3. Сначала на тех же примерах, а затем добавим новые:

In [4]:
testData2 = [
    pd.Series([10, 10, 10, 10, 10]),  # [False, False, False, False, False], не должно ничего измениться, по сравнению с примером выше, т.к. здесь нет аномалий.
    pd.Series([1, 10, 10, 10, 10]),  # [True, False, False, False, False], нет изменений, т.к. здесь один аномальный элемент.
    pd.Series([1, 5, 10, 10, 10]),  # [False, False, False, False, False], а вот тут первые два элемента уже не считаются аномалией.
    pd.Series([1, 5, 1, 1, 1]),  # [False, True, False, False, False], нет изменений, т.к. здесь один явно выраженный аномальный элемент.
    # Новые числовые серии:
    pd.Series([1, 10, 10, 1, 10, 1]),  # [True, False, False, False, False, False]
    pd.Series([1, 10, 10, 10, 10, 1]),  # [True, False, False, False, False, True]
    pd.Series([1, 1, 1, 10, 10, 10]),  # [False, False, False, False, False, False]
]
for i, test in enumerate(testData2):
    print("Input series {}:  {}".format(i + 1, list(test)))
    print("Output series {}: {}".format(i + 1, list(HampelFilter(test, window=3))))

Input series 1:  [10, 10, 10, 10, 10]
Output series 1: [False, False, False, False, False]
Input series 2:  [1, 10, 10, 10, 10]
Output series 2: [True, False, False, False, False]
Input series 3:  [1, 5, 10, 10, 10]
Output series 3: [False, False, False, False, False]
Input series 4:  [1, 5, 1, 1, 1]
Output series 4: [False, True, False, False, False]
Input series 5:  [1, 10, 10, 1, 10, 1]
Output series 5: [True, False, False, False, False, False]
Input series 6:  [1, 10, 10, 10, 10, 1]
Output series 6: [True, False, False, False, False, True]
Input series 7:  [1, 1, 1, 10, 10, 10]
Output series 7: [False, False, False, False, False, False]


В первых двух и четвёртой серии изменений по сравнению с предыдущим примером нет. Обратите внимание на третью числовую серию. В ней первые два элемента, которые раньше считались как аномальные, теперь игнорируются. Посмотрите на предыдущий пример, где `window = 5`, на этот пример, где `window = 3`, и на формулы (1) и (2). Подумайте и попробуйте объяснить самостоятельно, почему так произошло?

Ответив на этот вопрос, по аналогии вы сможете понять, почему в пятой числовой серии получился только один аномальный элемент, хотя кажется, что должны быть ещё. А также, почему при достаточно малом размере окна не удалось выявить аномалии в седьмом ряду. Кстати, их не получится выявить и при других размерах окна. Подставьте последовательно `window = 2, 4, 5, 6` и убедитесь в этом самостоятельно. Почему в седьмом числовом ряду нет аномалии?

Попробуйте также поэкспериментировать с другими двумя параметрами `sigma` и `scaleFactor` на тех же примерах, и посмотрите как изменится результат.

## Поиск индекса первого аномального элемента

На практике часто бывает нужно определить только первый аномальный или первый среди наибольших в числовом ряду элемент. Для этого можно использовать Python-реализацию функции [`HampelAnomalyDetection()`](https://tim55667757.github.io/TKSBrokerAPI/docs/tksbrokerapi/TradeRoutines.html#HampelAnomalyDetection) из библиотеки [`TKSBrokerAPI`](https://github.com/Tim55667757/TKSBrokerAPI). Эта функция возвращает минимальный индекс элемента в списке найденных аномалий или индекс первого максимального элемента во входном ряду, если его индекс меньше индекса аномального элемента. Если в числовом ряду нет аномалий или все элементы равны (нет максимумов), то возвращается None.

Если вы поняли, как работает предыдущая функция `HampelFilter()` и как она фильтрует аномалии, то в примерах ниже вам будет легко понять, почему возвращается тот или иной индекс элемента в качестве аномального. Но нужно помнить, что здесь учитывается наличие и местоположение максимальных элементов ряда. Попробуйте объяснить следующие примеры и ответы:

In [5]:
testData3 = [
    pd.Series([1, 1, 1, 1, 111, 1]),  #4
    pd.Series([1, 1, 10, 1, 1, 1]),  #2
    pd.Series([111, 1, 1, 1, 1, 1]),  #0
    pd.Series([111, 1, 1, 1, 1, 111]),  #0
    pd.Series([1, 11, 1, 111, 1, 1]),  #1
    pd.Series([1, 1, 1, 111, 99, 11]),  #3
    pd.Series([-111, 1, 1, 1, 1]),  #0
    pd.Series([1, 2, 1, -1, 1]),  #1
    pd.Series([1]),  #None
    pd.Series([1, 2]),  #None
    pd.Series([1, 1, 1, 1, 1, 1]),  #None
]
for i, test in enumerate(testData3):
    print("Input series {}: {}. Output index: {}".format(i + 1, list(test), HampelAnomalyDetection(test)))

Input series 1: [1, 1, 1, 1, 111, 1]. Output index: 4
Input series 2: [1, 1, 10, 1, 1, 1]. Output index: 2
Input series 3: [111, 1, 1, 1, 1, 1]. Output index: 0
Input series 4: [111, 1, 1, 1, 1, 111]. Output index: 0
Input series 5: [1, 11, 1, 111, 1, 1]. Output index: 1
Input series 6: [1, 1, 1, 111, 99, 11]. Output index: 3
Input series 7: [-111, 1, 1, 1, 1]. Output index: 0
Input series 8: [1, 2, 1, -1, 1]. Output index: 1
Input series 9: [1]. Output index: None
Input series 10: [1, 2]. Output index: None
Input series 11: [1, 1, 1, 1, 1, 1]. Output index: None


Как вы думаете, как изменятся ответы функции `HampelAnomalyDetection()` при подстановке других значений в параметрах `window`, `sigma` и `scaleFactor`? Поэкспериментируйте на тех же примерах и проверьте свои предположения.

## Фильтрация биржевых цен на аномалии

Рассмотрим фильтрацию по Хампелю на примере практической задачи выявления аномалий в ценовом ряду. Для получения нужного нам ряда с выбросами воспользуемся библиотекой для генерации тестовых биржевых данных **[PriceGenerator](https://github.com/Tim55667757/PriceGenerator/blob/master/README_RU.md)**.

Обычно трейдеры и аналитики используют ценовую модель в виде временного ряда OHLCV-candlesticks (open, high, low, close, volume), так называемых японских свечей. Одна строка таких данных представляет собой набор цен для построения одной свечи: дата открытия, цена открытия, наибольшая цена, наименьшая цена, цена закрытия на данном временном интервале и значение объёма торгов.

PriceGenerator — это простая библиотека, которую можно использовать как модуль Python или запускать из командной строки и генерировать случайные ценовые данные, максимально похожие на "настоящие цены", но с заранее заданными статистическими характеристиками. В библиотеке PriceGenerator можно задать интервал цен, таймфрейм, максимальное и минимальное значения для диапазона цен, максимальный размер для свечей, вероятность направления для очередной свечи, вероятность ценовых выбросов, количество генерируемых свечей и некоторые другие параметры.

Давайте сгенерируем ценовой ряд со следующими характеристиками:
- цены на графике состоят только из целых чисел (для простоты);
- интервал свечей: 1 день;
- горизонт генерации: 75 свечей;
- максимальная цена close: не более 140;
- минимальная цена close: не менее 40;
- начать ряд с цены close: 50;
- максимальный выброс "хвостов" свечей: 35;
- максимальный размер "тела" свечей: 15;
- вероятность того, что очередная генерируемая свеча будет вверх: 51%;
- вероятность того, что очередная генерируемая свеча будет иметь "выброс" вверх или вниз: 10%;
- общий тренд: пусть сначала будет растущий (40 свечей), затем падающий (10 свечей), затем опять растущий тренд (25 свечей).

Если вы запускаете PriceGenerator в консоли, для получения указанных характеристик ряда нужно указать параметры:

`pricegenerator --debug-level 10 --ticker "TEST_PRICES" --precision 0 --timeframe 1440 --horizon 75 --max-close 140 --min-close 40 --init-close 50 --max-outlier 35 --max-body 15 --max-volume 400000 --up-candles-prob 0.51 --outliers-prob 0.1 --trend-deviation 0.005 --split-trend "/\/" --split-count 40 10 25 --generate --render-google index.html`

Или можно использовать следующий код на Python:

In [6]:
uLogger.setLevel(0)  # Отключаем стандартное логирование модуля PriceGenerator, чтобы не путать их с нашими данными

# --- Инициализируем экземпляр класса генератора и настраиваем некоторые параметры:
priceModel = PriceGenerator()
priceModel.ticker = "TEST_PRICES"  # Произвольное имя (тикер) генерируемых цен.
priceModel.precision = 0  # Сколько знаков после запятой должно быть в сгенерированных ценах, по умолчанию 2.
priceModel.timeframe = timedelta(days=1)  # Временной интервал между генерируемыми свечами, 1 час по умолчанию.
priceModel.timeStart = datetime.today()  # С какой даты начинать генерацию свечей, по умолчанию с текущего времени.
priceModel.horizon = 75  # Сколько сгенерировать свечей, их должно быть не меньше 5-ти, по умолчанию 100.
priceModel.maxClose = 140  # Максимальная цена закрытия свечи во всей цепочке цен. Больше этого числа цена close любой свечи
                           # быть не может. По умолчанию число генерируется случайно в интервале (70, 90).
priceModel.minClose = 40  # Минимальная цена закрытия свечи во всей цепочке цен. Меньше этого числа цена close любой свечи
                           # быть не может. По умолчанию генерируется случайно в интервале (60, 70).
priceModel.initClose = 50  # Если этот параметр указан, то цена будет ценой close как бы "предыдущей" свечи.
                            # и одновременно ценой open самой первой свечи в генерируемой цепочке.
                            # None (по умолчанию) означает, что цена open первой свечи будет сгенерирована случайно
                            # в интервале (minClose, maxClose).
priceModel.maxOutlier = 35  # Максимальное значение для ценовых выбросов "хвостов" у свечей.
                            # None (по умолчанию) означает, что выбросы будут не более чем на (maxClose - minClose) / 10.
priceModel.maxCandleBody = 15  # Максимальное значение для размера тел свечей abs(open - close).
                               # None по умолчанию означает, что тело свечи может быть не более чем
                               # 90% от размера максимального выброса: maxOutlier * 90%.
priceModel.maxVolume = 400000  # Максимальное значение объёма торгов для одной свечи.
                               # По умолчанию берётся случайным образом из интервала (0, 100000).
priceModel.upCandlesProb = 0.51  # Вероятность того, что очередная генерируемая свеча будет вверх, 50% по умолчанию.
priceModel.outliersProb = 0.1  # Вероятность того, что очередная генерируемая свеча будет иметь "выброс", 3% по умолчанию.
priceModel.trendDeviation = 0.005  # Колебание цен close первой и последней свечей для определения тренда.
                                   # "NO trend" если разница меньше этого значения. По умолчанию ±0.005 или ±0.5%.
priceModel.trendSplit = "/\/"  # Пусть сначала будет растущий, затем падающий, затем опять растущий тренд.
priceModel.splitCount = [40, 10, 25]  # Число свечей в каждом тренде.

Запускаем генератор цен. После генерации они будут сохранены в переменной `priceModel.prices` в формате Pandas DataFrame:

In [10]:
priceModel.Generate()
priceModel.prices  # Сгенерированный Pandas DataFrame со столбцами "datetime", "open", "high", "low", "close" и "volume".

,datetime,open,high,low,close,volume
0,2023-01-01 13:03:40.457877+03:00,50.0,56.0,49.0,56.0,226289
1,2023-01-02 13:03:40.457877+03:00,56.0,56.0,55.0,55.0,220720
2,2023-01-03 13:03:40.457877+03:00,55.0,57.0,54.0,57.0,319989
3,2023-01-04 13:03:40.457877+03:00,57.0,63.0,56.0,61.0,352994
4,2023-01-05 13:03:40.457877+03:00,61.0,81.0,54.0,76.0,354851
...,...,...,...,...,...,...
70,2023-03-12 13:03:40.457877+03:00,109.0,111.0,95.0,97.0,137348
71,2023-03-13 13:03:40.457877+03:00,97.0,97.0,79.0,83.0,262121
72,2023-03-14 13:03:40.457877+03:00,83.0,86.0,68.0,69.0,208845
73,2023-03-15 13:03:40.457877+03:00,69.0,109.0,56.0,82.0,174972


Можно сохранить сгенерированные OHLCV-свечи в .csv-файл:

In [11]:
priceModel.SaveToFile(fileName="test.csv")

В PriceGenerator также можно сохранить график цен в виде html-файла. Метод `RenderGoogle()` создаёт статический график, а метод `RenderBokeh()` — интерактивный. Графики можно открыть здесь, в браузере, если указать параметр `viewInBrowser=True`. А интерактивный график Bokeh также можно отобразить в ячейке ноутбука, если указать параметр `inline=True`. Давайте получим представление о сгенерированных свечах:

In [9]:
priceModel.RenderGoogle(fileName="googlechart.html", viewInBrowser=False, title="Detect Anomalies With Hampel Method")
# TODO:
priceModel.RenderBokeh(fileName="bokehchart.html", viewInBrowser=False, title="Detect Anomalies With Hampel Method", showStatOnChart=False, showControlsOnChart=False, inline=True)

TypeError: RenderBokeh() got an unexpected keyword argument 'title'

После отработки методов `RenderGoogle()` и `RenderBokeh()` также будет автоматически пересчитана статистика ряда и сохранена в Markdown-файле с именем `fileName + ".md"`. Она станет доступна в переменной `priceModel.stat` и может пригодиться для дальнейших исследований.

In [ ]:
priceModel.stat

Кроме того, в Pandas Dataframe `priceModel.prices` добавляются новые столбцы со значением рассчитанных средних точек, размером свечей (дельта, разница между high и low) и индикаторов:

In [ ]:
priceModel.prices

На этом завершим знакомство с библиотекой [PriceGenerator](https://github.com/Tim55667757/PriceGenerator/blob/master/README_RU.md). При необходимости можно самостоятельно продолжить её изучение и запускать с различными параметрами. Для нашего примера, самое главное, что она сгенерировала Pandas Dataframe с данными, очень похожими на биржевые цены.

Как видно невооружённым глазом, в ряду на графике цен явно присутствуют выбросы — сверху и снизу у некоторых свечей. Нас будут интересовать не все такие выбросы, а только слишком длинные хвосты (тени свечей) или слишком большие размеры свечи в целом.

Далее очистим датафрейм с OHLCV-свечами и оставим только ценовые столбцы (без значений индикаторов), а также дельту свечей. Затем посчитаем размеры верхней (upper) и нижней (lower) теней свечей. Эти данные пригодятся нам для поиска выбросов вверх и вниз, а также для выявления аномального размера свечи.

In [ ]:
priceModel.prices = pd.DataFrame.copy(priceModel.prices, deep=True)[["datetime", "open", "high", "low", "close", "volume", "delta"]]
priceModel.prices["upper"] = priceModel.prices.high - priceModel.prices[["open", "close"]].max(axis=1)
priceModel.prices["lower"] = priceModel.prices[["open", "close"]].min(axis=1) - priceModel.prices.low
priceModel.prices

Изучив последние три получившихся столбца в датафрейме `priceModel.prices` убеждаемся, что теперь аномальные элементы, ранее обнаруженные визуально на графике, присутствуют в виде элементов числовых рядов. Эти ряды уже можно отфильтровать методом `HampelFilter()`. Давайте посмотрим, как фильтр Хампеля с параметрами по умолчанию (`window = 5`, `sigma = 3` и `scaleFactor = 1.4826`) справится с обнаружением аномальных ценовых выбросов и размеров. Как было показано ранее, в примерах выше, аномальными могут быть признаны как очень маленькие элементы, так и очень большие (по сравнению с остальными элементами в скользящем окне).

Фильтруем свечи с аномальными размерами:

In [ ]:
fDelta = HampelFilter(priceModel.prices.delta)

Фильтруем свечи с аномальными верхними тенями:

In [ ]:
fUpper = HampelFilter(priceModel.prices.upper)

Фильтруем свечи с аномальными нижними тенями:

In [ ]:
fLower = HampelFilter(priceModel.prices.lower)

Промаркируем аномальные свечи и тени на графике и посмотрим на результаты фильтрации:

In [ ]:
# TODO:
priceModel.RenderBokeh(
    fileName="hampel_default.html",
    viewInBrowser=False,  # Не открываем график в браузере лишний раз.
    darkTheme=False,  # Выбираем светлую тему графика.
    layouts=None,  # Пока нет дополнительных маркеров для графика.
    title="Detect Anomalies With Hampel Method",  # Заголовок графика.
    width=1800,  # Ширина графика в пикселях.
    height=940,  # Высота графика в пикселях.
    showStatOnChart=False,  # Не отображаем на графике лишний блок со статистикой.
    showControlsOnChart=True,  # Добавим на график кнопки для включения / выключения объектов.
    inline=True,  # Отобразим график в ячейке ноутбука.
)

На первый взгляд кажется, что аномалий найдено слишком много. Для наблюдателя, который не знает про малый размер скользящего окна, эти результаты будут неочевидны. Давайте попробуем расширить окно на весь числовой ряд и заново промаркировать аномальные элементы.

In [ ]:
fDelta = HampelFilter(priceModel.prices.delta, window=len(priceModel.prices.delta))
fUpper = HampelFilter(priceModel.prices.upper, window=len(priceModel.prices.delta))
fLower = HampelFilter(priceModel.prices.lower, window=len(priceModel.prices.delta))

# TODO:
priceModel.RenderBokeh(
    fileName="hampel_with_wide_window.html",
    viewInBrowser=False,  # Не открываем график в браузере лишний раз.
    darkTheme=False,  # Выбираем светлую тему графика.
    layouts=None,  # Пока нет дополнительных маркеров для графика.
    title="Detect Anomalies With Hampel Method",  # Заголовок графика.
    width=1800,  # Ширина графика в пикселях.
    height=940,  # Высота графика в пикселях.
    showStatOnChart=False,  # Не отображаем на графике лишний блок со статистикой.
    showControlsOnChart=True,  # Добавим на график кнопки для включения / выключения объектов.
    inline=True,  # Отобразим график в ячейке ноутбука.
)

Вот теперь видим, что на графике промаркированы только те элементы, про которые большинство наблюдателей, оценивающих картину в целом, скорее всего сказали бы, что это аномалии.

## Выводы

На практике фильтр Хампеля оказывается чрезвычайно эффективным. Благодаря современным библиотекам, таким как [Pandas](https://pandas.pydata.org/), он справляется с достаточно большими числовыми рядами на десятки тысяч элементов за секунды, а также легко поддаётся оптимизации и распараллеливанию (с помощью [CUDA Python](https://developer.nvidia.com/cuda-python) и декоратора [@cuda.jit от Numba](https://numba.readthedocs.io/en/stable/user/jit.html#compiling-python-code-with-jit), либо [Python Multiprocessing ThreadPool](https://superfastpython.com/threadpool-python/)).

Таким образом, если у вас стоит задача поиска аномалий в числовом ряду, попробуйте посмотреть в сторону их фильтрации методом Хампеля. Он даёт быстрые результаты, а также прост в понимании и реализации.

## Источники

1. Hampel F. R. The influence curve and its role in robust estimation. Journal of the American Statistical Association, 69, 382–393, 1974.
2. Hancong Liu, Sirish Shah and Wei Jiang. [On-line outlier detection and data cleaning](https://sites.ualberta.ca/~slshah/files/on_line_outlier_det.pdf). Computers and Chemical Engineering. Vol. 28, March 2004, pp. 1635–1647.
3. Lewinson Eryk. [Outlier Detection with Hampel Filter](https://towardsdatascience.com/outlier-detection-with-hampel-filter-85ddf523c73d). September 26, 2019.

## Исходный код модулей, используемых в примерах

- **[PriceGenerator](https://github.com/Tim55667757/PriceGenerator)** — этот модуль умеет генерировать временные ряды с данными, похожими на случайные биржевые цены с аномалиями.
- **[TKSBrokerAPI](https://github.com/Tim55667757/TKSBrokerAPI)** — этот модуль содержит в себе библиотеку TradeRoutines с необходимыми функциями.
  - **[TradeRoutines](https://github.com/Tim55667757/TKSBrokerAPI/blob/develop/tksbrokerapi/TradeRoutines.py)** — в этой библиотеке есть методы для анализа числовых рядов на аномалии.